In [1]:
import tensorflow as tf

import energy_py
from energy_py.agents import DQN, tfValueFunction
from energy_py.envs import BatteryEnv

Using TensorFlow backend.


In [2]:
EPISODE_LENGTH = 48
BRAIN_PATH = '/'
BATCH_SIZE = 10
EPISODES = 10
DISCOUNT = 0.9

total_steps = EPISODES * EPISODE_LENGTH

In [3]:
env = BatteryEnv(data_path='/mnt/c/Users/Adam Green/Documents/GitHub/energy_py/energy_py/experiments/battery/', 
                 episode_length=10)

In [4]:
agent = DQN(env, 
            DISCOUNT, 
            brain_path='/',
            Q=tfValueFunction,
            batch_size=BATCH_SIZE,
            total_steps=total_steps)

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #  initialize before starting episode
    done, step, total_step = False, 0, 0
    episode = 1
    observation = env.reset(episode)

    #  while loop runs through a single episode
    while done is False:
        #  select an action
        action = agent.act(sess=sess, obs=observation)
        #  take one step through the environment
        next_observation, reward, done, info = env.step(action)
        #  store the experience
        agent.memory.add_experience(observation, action, reward,
                                    next_observation, done, step, episode)
        step += 1
        total_step += 1
        observation = next_observation
        pred = sess.run(agent.Q_actor.prediction, {agent.Q_actor.obs:observation})
        if step == 9:
            #  with DQN we can learn within episode 
            #  get a batch to learn from
            batch = agent.memory.get_random_batch(BATCH_SIZE)

            train_info = agent.learn(sess=sess, batch=batch)

            if total_step % agent.update_target_net == 0:
                agent.update_target_network(sess)


shapes of arrays used in learning
obs shape (9, 6)
actions shape (9, 2)
rews shape (9,)
terminal shape (9,)
[ 0.27893147  0.31527922  0.35447797  0.29779416  0.27893147  0.22632609
  0.29779416  0.22632609  0.34080687]
[False False False False False False False False False]
max_q after termnal [ 0.27893147  0.31527922  0.35447797  0.29779416  0.27893147  0.22632609
  0.29779416  0.22632609  0.34080687]
before bellman eqn
rews shape (9,)
max_q shape (9,)
targets shape (9,)
action indicies shape (9, 1)


InvalidArgumentError: Incompatible shapes: [900,1] vs. [9,1]
	 [[Node: Q_actor/huber_loss/Sub = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Q_actor/Reshape, _arg_Q_actor/target_0_2)]]

Caused by op 'Q_actor/huber_loss/Sub', defined at:
  File "/home/adam/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/adam/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/adam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/adam/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/adam/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/adam/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZM